## Scraping Craigslist forums

* using keywords related to __scams__


In [131]:
import requests
from bs4 import BeautifulSoup
import re
from IPython.display import HTML

In [1]:
kws = '''Scam, scams, scammed, scamming, scammer, scammers; 
        fraud, frauds, defrauded, fraudster, fraudsters; 
        con, cons, conned, conning, con artist, con artists; 
        trick, tricks, tricked, tricking, trickster, tricksters; 
        swindle, swindles, swindled, swindler, swindlers'''

In [47]:
kw_list=[i.lower() for k in kws.strip().split(';') for i in k.strip().split(', ')]

In [49]:
print(' OR '.join(kw_list))

scam OR scams OR scammed OR scamming OR scammer OR scammers OR fraud OR frauds OR defrauded OR fraudster OR fraudsters OR con OR cons OR conned OR conning OR con artist OR con artists OR trick OR tricks OR tricked OR tricking OR trickster OR tricksters OR swindle OR swindles OR swindled OR swindler OR swindlers


### QUESTION: what is the search syntax for the Criagslist forum

* using commas? e.g. scam,scams,scamming
* boolean? e.g. scam OR scams OR scamming

Both return results but it looks like they are doing a logical AND

So might need to search each keyword in turn

In [ ]:
# search url
# https://forums.craigslist.org/?act=searchFrameMaker&SQ=scam%2C+scams%2C+scamming&forumID=8

# this will get the frame version of a search
# but we want the second row left hand pane which looks like you can get with act=RSR

In [70]:
url2 = 'https://forums.craigslist.org/?act=RSR&forumID=8&SQ=scam&&resultsPerPage=100'
r2=requests.get(url2)
s2=BeautifulSoup(r2.text)

### Anatomy of the results frame

```
    <section class="searchresults">
        <h1>results for your search for "..."</h1>
        <div class="paginator">
            <div class="prev"></div>
            <div class="next"></div>
        </div>
    
        <article class="searchresult normal">
        ....
        </article>
        
        ...
        <article class="searchresult normal">
        </article>
        
    </section>
```

In [71]:
results = s2.body.find('section', {'class': 'searchresults'})
results.contents[:5]

['\n',
 <h1>results for your search for "scam":</h1>,
 '\n',
 <div class="paginator">
 <div class="prev"></div><div class="next"> <a href="?act=RSR&amp;forumID=8&amp;offset=100&amp;resultsPerPage=100&amp;SQ=scam">next 100 &gt;&gt;</a>
 </div></div>,
 '\n']

In [76]:
paging = results.find('div', {'class':'paginator'})
paging

<div class="paginator">
<div class="prev"></div><div class="next"> <a href="?act=RSR&amp;forumID=8&amp;offset=100&amp;resultsPerPage=100&amp;SQ=scam">next 100 &gt;&gt;</a>
</div></div>

In [77]:
has_next = paging.find('div',{'class': 'next'}).findAll('a')
if len(has_next)>0:
    print('next page of results is:', has_next[0])

next page of results is: <a href="?act=RSR&amp;forumID=8&amp;offset=100&amp;resultsPerPage=100&amp;SQ=scam">next 100 &gt;&gt;</a>


In [120]:
has_next[0].attrs['href']

'?act=RSR&forumID=8&offset=100&resultsPerPage=100&SQ=scam'

In [74]:
result_list=results.findAll('article',{'class':'searchresult'})
print('current page has {} results for search term'.format(len(result_list)))

current page has 100 results for search term


In [79]:
result_list[2]

<article class="searchresult normal">
<header class="metadata"><span class="area"><span class="area" title="new haven, CT">hvn</span></span> <span class="title"><a href="?ID=278916777" target="_top">I wonder what fraction of emails sent there</a></span> &lt; <span class="handle hnd">HelpfulHedda</span> &gt; <span class="date">2017-02-11 09:42</span></header>
<p class="body">were appropriate!

What's wrong with the contact form?    Just how many "tools" does it take to replace a light bulb?   You can even bookmark the popup for a particular form option (save the URL in your browser) to simplify/expedite repeated submissions.Here are the likely appropriate choices on that form page:

scams, spam, flagging

    Someone is spamming or overposting on craigslist
    I encountered a fraudulent posting or a scam on craigslist
    I need to report posts that are prohibited, or violate the TOU
</p>
</article>

In [80]:
for r in result_list:
    print(r.find('a'))

<a href="?ID=278970264" target="_top">Dunno where you're at but</a>
<a href="?ID=278957414" target="_top">Hollow victory, till rampant scam ads are gone</a>
<a href="?ID=278916777" target="_top">I wonder what fraction of emails sent there</a>
<a href="?ID=278880137" target="_top">More BULLSHIT!!!</a>
<a href="?ID=278834210" target="_top">Suggest limiting number of auto and truck post</a>
<a href="?ID=278818879" target="_top">This forum is not for reports about</a>
<a href="?ID=278818814" target="_top">You are joking right?</a>
<a href="?ID=278818653" target="_top">This forum is for feedback about</a>
<a href="?ID=278818603" target="_top">Possible scam</a>
<a href="?ID=278749985" target="_top">Here is a start on tracking that down</a>
<a href="?ID=278742449" target="_top">Posting other people's phone numbers</a>
<a href="?ID=278615128" target="_top">Scam Donald Trump ad not removed</a>
<a href="?ID=278563635" target="_top">its because you put your number in the ad</a>
<a href="?ID=27856

# Scratchpad

In [11]:
r = requests.get('https://forums.craigslist.org/?act=searchFrameMaker&SQ=scam%2C+scams%2C+scamming&forumID=8')

In [14]:
s1 = BeautifulSoup(r.text)

In [15]:
s1

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Frameset//EN" "http://www.w3.org/TR/html4/frameset.dtd">
<html>
<head>
<title>cl - feedback - forums - craigslist</title>
<script src="//www.craigslist.org/js/jquery-1.11.3.min.js?v=80826eb702c0268c524f4431244c9556" type="text/javascript"></script>
<script src="//www.craigslist.org/js/general-concat.min.js?v=f69d411b2b700284dd519993fdf5e4b9" type="text/javascript"></script>
<script src="//www.craigslist.org/js/forums.min.js?v=1286806026d0667c8902c0826a2c0cbc" type="text/javascript"></script>
<!--[if lt IE 9]>
<script src="//www.craigslist.org/js/html5shiv.min.js?v=096822b653643ed1af3136947e4ea79a" type="text/javascript" ></script>

<script src="//www.craigslist.org/js/respond-fork.min.js?v=d7e1cb0d97ee0c0c9d84a7d4f1d03469" type="text/javascript" ></script>

<![endif]-->
<!--[if lte IE 7]>
<script src="//www.craigslist.org/js/json2.min.js?v=178d4ad319e0e0b4a451b15e49b71bec" type="text/javascript" ></script>

<![endif]-->
</head>
<frameset row

In [81]:
r

<article class="searchresult normal">
<header class="metadata"><span class="area"><span class="area" title="jackson, TN">jxt</span></span> <span class="title"><a href="?ID=275575701" target="_top">What did the first email say?</a></span> &lt; <span class="handle hnd">MyThreeCents</span> &gt; <span class="date">2016-10-15 19:39</span></header>
<p class="body">If this is the 41st person trying to scam you....assess each one, there is usually something they have in common from the way they start off. It's automated and then once yu reply they know they have a live one....learn not to take the bait.</p>
</article>

In [82]:
url2 = 'https://forums.craigslist.org/?act=Q&ID=275575701'

In [83]:
r2=requests.get(url2)
s2=BeautifulSoup(r2.text)

In [84]:
s2

<!DOCTYPE html>
<html>
<head>
<title>forums - craigslist</title>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<link href="//www.craigslist.org/styles/craigslist.css?v=94814967e84c2ff11c6abefe670882db" media="all" rel="stylesheet" type="text/css"/><link href="//www.craigslist.org/styles/forums-common.css?v=52755766f6b3ed3201f29944ac61f07b" media="all" rel="stylesheet" type="text/css"/>
<script src="//www.craigslist.org/js/jquery-1.11.3.min.js?v=80826eb702c0268c524f4431244c9556" type="text/javascript"></script>
<script src="//www.craigslist.org/js/general-concat.min.js?v=f69d411b2b700284dd519993fdf5e4b9" type="text/javascript"></script>
<script src="//www.craigslist.org/js/forums.min.js?v=1286806026d0667c8902c0826a2c0cbc" type="text/javascript"></script>
<!--[if lt IE 9]>
<script src="//www.craigslist.org/js/html5shiv.min.js?v=096822b653643ed1af3136947e4ea79a" type="text/javascript" ></script>

<script src="//www.craigslist.org/js/respond-fork.min.js?v=d7e1cb0d97e

In [85]:
s2.body.find('div', {'class':'normal'})

<div class="normal">
<div id="titleLine">
<span><b>What did the first email say?</b></span>
<span>&lt; <span class="handle hnd">MyThreeCents</span> &gt;</span>
<time>2016-10-15 19:39</time>
<br/>
<br/>
</div>
<span class="quote">If this is the 41st person trying to scam you....assess each one, there is usually something they have in common from the way they start off. It's automated and then once yu reply they know they have a live one....learn not to take the bait.</span>
<br/>
<br/>
<a class="contextlink" href="https://forums.craigslist.org/?ID=275575701" target="_top">https://forums.craigslist.org/?ID=275575701</a>
</div>

In [86]:
r

<article class="searchresult normal">
<header class="metadata"><span class="area"><span class="area" title="jackson, TN">jxt</span></span> <span class="title"><a href="?ID=275575701" target="_top">What did the first email say?</a></span> &lt; <span class="handle hnd">MyThreeCents</span> &gt; <span class="date">2016-10-15 19:39</span></header>
<p class="body">If this is the 41st person trying to scam you....assess each one, there is usually something they have in common from the way they start off. It's automated and then once yu reply they know they have a live one....learn not to take the bait.</p>
</article>

In [89]:
r3=requests.get(url2.replace('=Q','=showThread'))
s3=BeautifulSoup(r3.text)

In [121]:
s3

<!DOCTYPE html>
<html>
<head>
<title>forums - craigslist</title>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<link href="//www.craigslist.org/styles/craigslist.css?v=94814967e84c2ff11c6abefe670882db" media="all" rel="stylesheet" type="text/css"/><link href="//www.craigslist.org/styles/forums-common.css?v=52755766f6b3ed3201f29944ac61f07b" media="all" rel="stylesheet" type="text/css"/>
<script src="//www.craigslist.org/js/jquery-1.11.3.min.js?v=80826eb702c0268c524f4431244c9556" type="text/javascript"></script>
<script src="//www.craigslist.org/js/general-concat.min.js?v=f69d411b2b700284dd519993fdf5e4b9" type="text/javascript"></script>
<script src="//www.craigslist.org/js/forums.min.js?v=1286806026d0667c8902c0826a2c0cbc" type="text/javascript"></script>
<!--[if lt IE 9]>
<script src="//www.craigslist.org/js/html5shiv.min.js?v=096822b653643ed1af3136947e4ea79a" type="text/javascript" ></script>

<script src="//www.craigslist.org/js/respond-fork.min.js?v=d7e1cb0d97e

In [123]:
tp=s3.body.find('p', {'class': 'singlethreadheader'})

In [134]:
re.search('in thread (\d+) ',tp.text).group(1)

'137702'

In [124]:
thread=s3.body.find('div',{'class':'threads'})

In [127]:
thread.insert(0,tp)

In [128]:
thread

<div class="threads"><p class="singlethreadheader">
    post 275575701 is located in thread 137702 of <a href="?forumID=8" target="_top">cl - feedback</a>
</p>
<article class="thread"><div class="threadline normal first"> <span class="area" title="jackson, TN">jxt</span> <a class="title" href="?act=Q&amp;ID=275566908" target="R">Website Scam</a> &lt; <span class="handle hnd">liontamer1000</span> &gt; <time>2016-10-15 11:59</time><span class="ratings"><span class="minus">-14</span><span class="plus">+3</span></span> </div><div class="threadline normal"> <span class="dotz"> : . . </span> <a class="title" href="?act=Q&amp;ID=275569512" target="R">staff of CL provide you with easy area to</a> &lt; <span class="handle hnd">cl-VPD</span> &gt; <time>2016-10-15 14:04</time><span class="ratings"><span class="minus">-9</span></span> <span class="R">link</span> </div><div class="threadline normal"> <span class="dotz"> : . .  : . . </span> <a class="title" href="?act=Q&amp;ID=275572600" target="R"

In [22]:
r3=requests.get('https://forums.craigslist.org/?act=RSR&forumID=8&resultsPerPage=100&SQ=scam%2C%20scams%2C%20scamming')

In [23]:
s3=BeautifulSoup(r3.text)

In [35]:
len(s3.body.findAll('article',{'class': 'searchresult'}))

24

## Reformating the thread view

* seems like one `<article class="thread">` element per thread [*CONFIRM*]
* each post in the thread is at the same level in a `<div class="threadline normal">` element
* the first item in the thread has a `first` class attribute


In [98]:
print(s3.body.find('div',{'class':'threads'}).prettify()[:700])

<div class="threads">
 <article class="thread">
  <div class="threadline normal first">
   <span class="area" title="jackson, TN">
    jxt
   </span>
   <a class="title" href="?act=Q&amp;ID=275566908" target="R">
    Website Scam
   </a>
   &lt;
   <span class="handle hnd">
    liontamer1000
   </span>
   &gt;
   <time>
    2016-10-15 11:59
   </time>
   <span class="ratings">
    <span class="minus">
     -14
    </span>
    <span class="plus">
     +3
    </span>
   </span>
  </div>
  <div class="threadline normal">
   <span class="dotz">
    : . .
   </span>
   <a class="title" href="?act=Q&amp;ID=275569512" target="R">
    staff of CL provide you with easy area to
   </a>
   &lt;
   <spa


In [102]:
thread = s3.body.find('div',{'class': 'threads'})
posts_in_thread = thread.article.findAll('div',{'class': 'threadline'})
print('{} posts in this thread'.format(len(posts_in_thread)))

11 posts in this thread


In [109]:
print(posts_in_thread[5].prettify())

<div class="threadline normal">
 <span class="dotz">
  : . .  : . .  : . .  : . .  : . .
 </span>
 <a class="title" href="?act=Q&amp;ID=275576027" target="R">
  F&amp;S §
 </a>
 &lt;
 <span class="handle hnd">
  cl-VPD
 </span>
 &gt;
 <time>
  2016-10-15 19:55
 </time>
</div>



In [115]:
level=posts_in_thread[5].find('span',{'class':'dotz'}).text.count(':')
level

5